In [1]:
import sys
sys.path.append('..')

from abc import ABC, abstractmethod
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum
from sqlalchemy import or_, not_, func
from numbers import Number
import webbrowser

from src.database.session import SessionLocal
from src.pingers import *
from src.models import *
from src import crud
from src.abc.descriptive import DescriptiveDF
from src.utils import *
from src.models.enums.facebook.creative_features import BOOLEAN_TEXT_FEATURES
from src.utils import *
from src.models.enums.facebook import Target
from src.crud.utils.get_performance import get_conversion_rates_dict
from src.models.enums import Industry
from src.models.enums.facebook import TextType
db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=12)
shop_id = 44301396

# df = ping_facebook_creative_target_and_performance(db=db, shop_id=16038, start_date='2023-09-20')

2023-11-27 13:31:21.081 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [2]:
df = ping_ads_insights_all_platforms(db=db, get_industry=True)

SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS currency_exchange_rate_code, currency_exchange_rate.rate_from_usd AS currency_exchange_rate_rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC


2023-11-27 13:32:04.075 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: crm_dataset_dev.csv


In [19]:
df_ = df.groupby(['shop_id', 'platform'])[['spend_USD']].sum().unstack(level='platform').loc[:, 'spend_USD']

In [17]:
df_ =  df_.unstack(level=1).loc[:, 'spend_USD']

In [3]:
dt = date.today()
while dt > date(year=2015, month=1, day=1):
    print(dt)
    dt -= relativedelta(months=1)

2023-11-16
2023-10-16
2023-09-16
2023-08-16
2023-07-16
2023-06-16
2023-05-16
2023-04-16
2023-03-16
2023-02-16
2023-01-16
2022-12-16
2022-11-16
2022-10-16
2022-09-16
2022-08-16
2022-07-16
2022-06-16
2022-05-16
2022-04-16
2022-03-16
2022-02-16
2022-01-16
2021-12-16
2021-11-16
2021-10-16
2021-09-16
2021-08-16
2021-07-16
2021-06-16
2021-05-16
2021-04-16
2021-03-16
2021-02-16
2021-01-16
2020-12-16
2020-11-16
2020-10-16
2020-09-16
2020-08-16
2020-07-16
2020-06-16
2020-05-16
2020-04-16
2020-03-16
2020-02-16
2020-01-16
2019-12-16
2019-11-16
2019-10-16
2019-09-16
2019-08-16
2019-07-16
2019-06-16
2019-05-16
2019-04-16
2019-03-16
2019-02-16
2019-01-16
2018-12-16
2018-11-16
2018-10-16
2018-09-16
2018-08-16
2018-07-16
2018-06-16
2018-05-16
2018-04-16
2018-03-16
2018-02-16
2018-01-16
2017-12-16
2017-11-16
2017-10-16
2017-09-16
2017-08-16
2017-07-16
2017-06-16
2017-05-16
2017-04-16
2017-03-16
2017-02-16
2017-01-16
2016-12-16
2016-11-16
2016-10-16
2016-09-16
2016-08-16
2016-07-16
2016-06-16
2016-05-16

In [2]:
get_first_and_last_day_of_the_month(month='2023-01')

(datetime.date(2023, 1, 1), datetime.date(2023, 1, 31))

In [2]:
text_data = crud.fb_creative_features.get_text_sample(db=db, sample_size=100_000, start_date = start_date, end_date = end_date)
text_df = pd.DataFrame(text_data)
crm = db.query(Crm.shop_id, Crm.industry).filter(Crm.shop_id.in_(text_df.shop_id.unique().tolist())).all()
crm_df = pd.DataFrame(crm)
df = text_df.merge(crm_df)

TypeError: CRUDFacebookCreativeFeatures.get_text_sample() missing 1 required positional argument: 'text_type'

In [2]:
data = crud.fb_creative_features.get_text_sample(
    db=db,
    # industry=Industry.pet_supplies,
    text_type=TextType.description,
    start_date=date.today() - timedelta(days=900),
    end_date=date.today(),
)

In [4]:
len(data)

10000

In [3]:
len(data)

57

In [5]:
df = pd.DataFrame(data)
df

,shop_id,title,primary,description,industry_enum
0,119989244,[],[En høj kvalitet børne dunjakke er en fantasti...,[],clothing_and_accessories
1,10893054,[],[Mid-Year Summer Sale Up to 90%OFF ! Almost Go...,[Extra 30% OFF with Code: SALE30],clothing_and_accessories
2,10893054,[],[Final Clearance ! More Sales Items Add Into T...,[Extra 40% OFF Code: SALE40],clothing_and_accessories
3,116557323,[Dnevna doza mikronutrijenata],[🌟 Dnevna doza mikronutrijenata! Upoznaj naše ...,[],clothing_and_accessories
4,116557323,[Dnevna doza mikronutrijenata],[🌟 Dnevna doza mikronutrijenata! Upoznaj naše ...,[],clothing_and_accessories
...,...,...,...,...,...
9995,2,[We're in this together. ],[It's no secret cleaning products are walking ...,[Subscribe & Save 30%],clothing_and_accessories
9996,2,[We're in this together. ],[It's no secret cleaning products are walking ...,[Subscribe & Save 30%],clothing_and_accessories
9997,2,[$10 off $30+ with code JULY4TH],[Celebrate Independence Day by saving big on o...,[],clothing_and_accessories
9998,2,[Save $10 with code JULY4TH],"[This week only, save $10 when you spend $30+ ...",[ ],clothing_and_accessories


In [3]:
class E(str, Enum):
    a='b'

In [10]:
df = text_df.merge(crm_df)

In [11]:
df.industry.value_counts()

industry
Clothing and accessories           612
Health and beauty                  359
Unknown                            252
Food and drink                      99
Home and garden                     69
Electronics                         48
Pet supplies                        41
Art and entertainment               38
Sports and recreation               36
Toys and games                      25
Business equipment and supplies     24
Hardware and automotive              6
Name: count, dtype: int64

In [4]:
pd.DataFrame(data)

,title,primary,description
0,[],[عداد ساعات التشغيل\nللاستخدام في المولدات و ا...,[]
1,[],[Our timeless essentials - Bivale bracelets in...,[]
2,[],[],[]
3,[],[Our curated boxes are the perfect way to brin...,"[Mini closet refresh, Mystery Boxes for just $50]"
4,[],[Our curated boxes are the perfect way to brin...,[The fan-favorite Mystery Box comes with two (...
...,...,...,...
1826,[Try a Starter Pack for $2.49!],[Our Premium Pads fit into your regular underw...,"[Trusted by 100,000+ happy customers.]"
1827,[Premier Supplements For Women!],[Our Women's Essentials Bundle provides the pe...,"[Trusted by 100,000+ happy customers.]"
1828,[SUMMER READY ],[Summer is here ☀️ Add Dainty's gold & sterlin...,[20% OFF CODE: LOVE20]
1829,[Get 10 OFF],[Time to reconsider board equipment? Take 10%...,[]


In [8]:
big_number_human_format(1_000)

'1.00K'

In [6]:
[[[], 1], [[1]], 2]

[[[], 1], [[1]], 2]

In [2]:
df = ping_facebook_creative_and_performance(db=db, shop_id = 16038)

In [3]:
df.discount_list

0           [10, 10]
1               [10]
2       [10, 10, 10]
3           [10, 10]
4           [10, 10]
            ...     
3651             [0]
3652             [0]
3653             [0]
3654             [0]
3655            [80]
Name: discount_list, Length: 3656, dtype: object

In [9]:
def test_finally():
    a = {'k': 1}
    try:
        return a
    finally:
        a['k'] -= 1
        print('kraj')

In [10]:
a = test_finally()
print(a['k'])
a['k'] += 2
print(a['k'])

kraj
0
2


: 

In [7]:
a

7

In [3]:
df = ping_ads_insights_all_platforms(db=db)

SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS currency_exchange_rate_code, currency_exchange_rate.rate_from_usd AS currency_exchange_rate_rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC


2023-11-06 12:47:36.400 | DEBUG    | src.pingers.ping_ads_insights:ping_ads_insights_by_platform:94 - db type: <class 'sqlalchemy.orm.session.Session'>
2023-11-06 12:48:13.770 | DEBUG    | src.pingers.ping_ads_insights:ping_ads_insights_by_platform:94 - db type: <class 'sqlalchemy.orm.session.Session'>
2023-11-06 12:48:28.869 | DEBUG    | src.pingers.ping_ads_insights:ping_ads_insights_by_platform:94 - db type: <class 'sqlalchemy.orm.session.Session'>


In [6]:
df.year_month.max()

Timestamp('2023-11-01 00:00:00')

In [3]:
eval(os.getenv('CUSTOM_ANALYSIS'))

True

In [6]:
eval('False') is False

True

In [2]:
df = crud.fb_daily_performance.get_performance(db=db, shop_id=shop_id, start_date = '2023-09-20', period=Period.date)

In [2]:
df = ping_facebook_creative(db=db, shop_id=16038)

In [3]:
len(df)

5417

In [4]:
# df = crud.fb_daily_performance.get_performance(db=db, shop_id=16038, start_date='2023-06-01')
df = ping_facebook_creative_target_and_performance(db=db, shop_id=16038, start_date='2023-06-01')

In [5]:
df.groupby(['ad_id'])['spend_USD'].sum().sort_values(ascending=False)

ad_id
23855091528590755    69121.32
23853108353540755    51533.70
23856746700950755    44520.76
23855200131640755    40054.91
23854607500280755    32254.04
                       ...   
23858727818090755       35.61
23857982741540755       25.05
23855968645130755       17.37
23857982741560755        2.43
23854131616400755        0.00
Name: spend_USD, Length: 119, dtype: float64

In [2]:
isinstance(True, Number)

True

In [2]:
webbrowser.open('https://discuss.streamlit.io/t/a-new-tab-doesnt-open-up-in-my-web-browser-if-i-run-the-streamlit-python-script/285/25')

True

In [4]:
isinstance(5, bool)

False

In [2]:
df = ping_target(db=db, shop_id=16038)

2023-08-30 07:28:52.771 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.781 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.788 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.788 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.789 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.790 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.790 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.791 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.791 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.792 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30 07:28:52.793 | DEBUG    | src.pingers.ping_target:deduce_audience:110 - tu sam
2023-08-30

In [3]:
df.audience.value_counts()

audience
Audience.broad          385
Audience.interest       138
Audience.retargeting     85
Audience.lookalike       42
Name: count, dtype: int64

In [4]:
df.loc[df.target == Target.remarketing, 'audience']

105    Audience.retargeting
109    Audience.retargeting
127    Audience.retargeting
128    Audience.retargeting
138    Audience.retargeting
               ...         
577    Audience.retargeting
609    Audience.retargeting
620    Audience.retargeting
626    Audience.retargeting
637    Audience.retargeting
Name: audience, Length: 85, dtype: object

In [4]:
from inspect import getfullargspec

def f(aha: int = 5):
    pass

getfullargspec(f).args

['aha']

In [113]:
class A(ABC):
    @print_execution_time
    @abstractmethod
    def am(self, **kwargs):
        pass

In [114]:
class B(A):
    def am(self):
        print('h')

In [115]:
b = B()
b.am()

h


In [88]:
df = ping_target_and_performance(db=db, shop_id=16038, enum_to_value=True)
df['n_ads'] = 1
# df = df[ftd.metric_columns + ftd.descriptive_columns + ['year_month', 'primary', 'description', '']]
df[ftd.fake_feature] = '.'

In [92]:
df.spend_USD.sum()

3612839.8100000005

In [98]:
df2 = df.copy()

In [99]:
for explode_col in ftd.explode_descriptive_columns:
    df[ftd.metric_columns] = df[ftd.metric_columns].div(df.age_groups.apply(len), axis=0)
    df = df.explode(explode_col).dropna(axis=0, subset=explode_col)

In [100]:
df2 = df2.explode('age_groups')

In [101]:
df2.spend_USD.sum() - df.spend_USD.sum()

-4.656612873077393e-10

In [102]:
df = ping_target_and_performance(db=db, shop_id=16038, enum_to_value=True)
df['n_ads'] = 1
df = df[ftd.metric_columns + ftd.descriptive_columns + ['year_month']]
df[ftd.fake_feature] = '.'
for explode_col in ftd.explode_descriptive_columns:
    df[ftd.metric_columns] = df[ftd.metric_columns].div(df.age_groups.apply(len), axis=0)
    df = df.explode(explode_col).dropna(axis=0, subset=explode_col)
stacked_df = df.set_index(list(set(df.columns) - set(ftd.descriptive_columns + [ftd.fake_feature]))).stack(level=-1).reset_index()
level_column = [col for col in stacked_df.columns if 'level' in str(col)][0]
stacked_df.rename(columns={level_column: 'feature', 0: 'feature_value'}, inplace=True)
shop_descriptive_df = stacked_df.groupby(['year_month', 'feature', 'feature_value'])[ftd.metric_columns].sum()

In [106]:
sumary_df[sumary_df.index.get_level_values('feature').isin([ftd.fake_feature])]

,,,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
year_month,feature,feature_value,,,,,,
2016-08-01,fake_feature,.,2984.80,279166.0,4834.0,45.0,0.00,41.0
2016-09-01,fake_feature,.,4722.21,664735.0,10510.0,92.0,0.00,62.0
2016-10-01,fake_feature,.,4929.56,818328.0,10611.0,221.0,0.00,74.0
2016-11-01,fake_feature,.,5200.03,973057.0,8421.0,151.0,0.00,74.0
2016-12-01,fake_feature,.,4140.95,624521.0,5817.0,77.0,0.00,60.0
...,...,...,...,...,...,...,...,...
2023-04-01,fake_feature,.,227635.62,13296255.0,126740.0,1546.0,307917.22,20.0
2023-05-01,fake_feature,.,143810.16,9219372.0,82268.0,1280.0,230801.32,21.0
2023-06-01,fake_feature,.,137206.08,9432753.0,83510.0,1175.0,189821.41,21.0


In [75]:
stacked_df = df.set_index(list(set(df.columns) - set(ftd.descriptive_columns + [ftd.fake_feature]))).stack(level=-1).reset_index()
level_column = [col for col in stacked_df.columns if 'level' in str(col)][0]
stacked_df.rename(columns={level_column: 'feature', 0: 'feature_value'}, inplace=True)

In [76]:
sumary_df = stacked_df.groupby(['year_month', 'feature', 'feature_value'])[ftd.metric_columns].sum()

spend_USD       impr   clicks  \
year_month feature         feature_value                                  
2016-08-01 creative_type   carousel         2984.80   279166.0   4834.0   
           cta             False            2714.21   264301.0   4539.0   
                           True              270.59    14865.0    295.0   
           discount        True             2984.80   279166.0   4834.0   
           emoji           False            2984.80   279166.0   4834.0   
...                                             ...        ...      ...   
2023-08-01 target          acquisition    123725.51  7482632.0  86006.0   
                           remarketing      7702.16   331759.0   1654.0   
           urgency         False          141439.29  8570155.0  95637.0   
           user_addressing True           141439.29  8570155.0  95637.0   
           weasel_words    False          141439.29  8570155.0  95637.0   

                                           purch  purch_value_USD  n_ads  
year_month feature         feature_value                                  
2016-08-01 creative_type   carousel         45.0             0.00     41  
           cta             False            44.0             0.00     35  
                           True              1.0             0.00      6  
           discount        True             45.0             0.00     41  
           emoji           False            45.0             0.00     41  
...                                          ...              ...    ...  
2023-08-01 target          acquisition    1085.0        201472.23     30  
                           remarketing     704.0        101519.60      9  
           urgency         False          1871.0        316236.94     43  
           user_addressing True           1871.0        316236.94     43  
           weasel_words    False          1871.0        316236.94     43  

[1741 rows x 6 columns]

In [72]:
[col for col in stacked_df.columns if '5' in col]

['year_month',
 'impr',
 'n_ads',
 'spend_USD',
 'purch_value_USD',
 'purch',
 'clicks',
 'level_7',
 0]

In [12]:
df.groupby(['year_month'])[ftd.metric_columns].sum()

,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
year_month,,,,,,
2016-08-01,2984.80,279166.0,4834.0,45.0,0.00,41
2016-09-01,4722.21,664735.0,10510.0,92.0,0.00,62
2016-10-01,4929.56,818328.0,10611.0,221.0,0.00,74
2016-11-01,5200.03,973057.0,8421.0,151.0,0.00,74
2016-12-01,4140.95,624521.0,5817.0,77.0,0.00,60
...,...,...,...,...,...,...
2023-04-01,227635.62,13296255.0,126740.0,1546.0,307917.22,20
2023-05-01,143810.16,9219372.0,82268.0,1280.0,230801.32,21
2023-06-01,137206.08,9432753.0,83510.0,1175.0,189821.41,21


In [13]:
df

,shop_id,account_id,ad_id,creative_type,title,primary,description,cta,discount,emoji,...,adset_id,spend,impr,clicks,purch,purch_value,year_month,spend_USD,purch_value_USD,n_ads
0,16038,923409341101559,23842502774150755,carousel,[],"[Fresh Fall Collections at FEW MODA, 10% OFF F...",[10% OFF Your First Order],False,True,False,...,23842502773950755,15.78,1697.0,21.0,2.0,0.00,2016-08-01,15.78,0.00,1
1,16038,923409341101559,23842502774270755,carousel,[],"[FEW MODA 2016 Fall Collections-Part I, Free S...",[10% OFF Your First Order],False,True,False,...,23842502773990755,7.25,284.0,5.0,0.0,0.00,2016-08-01,7.25,0.00,1
2,16038,923409341101559,23842502774380755,carousel,[],"[Fresh Fall Collections at FEW MODA, 10% OFF F...","[10% OFF Your First Purchase, 10% OFF Your Fir...",False,True,False,...,23842502773960755,15.48,1779.0,13.0,0.0,0.00,2016-08-01,15.48,0.00,1
3,16038,923409341101559,23842502774390755,carousel,[],[Summer Time Pleasure! 10% OFF Code: FEWMODASU...,[10% OFF with FEWMODASUMMERPLEASURE],False,True,False,...,23842502773990755,5.85,246.0,1.0,0.0,0.00,2016-08-01,5.85,0.00,1
4,16038,923409341101559,23842502774480755,carousel,[],"[Fresh Fall Collections at FEW MODA, 10% OFF F...",[10% OFF Your First Order],False,True,False,...,23842502773980755,18.53,2537.0,15.0,0.0,0.00,2016-08-01,18.53,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460,16038,923409341101559,23857024785990755,dynamic,[Change the Way You Shop],[Get summer-ready in styles with our high-qual...,[⭐⭐⭐⭐⭐],False,False,True,...,23856265061000755,45.27,895.0,7.0,5.0,528.00,2023-08-01,45.27,528.00,1
3461,16038,923409341101559,23857029567760755,dynamic,[Change the Way You Shop],[Get summer-ready in styles with our high-qual...,[⭐⭐⭐⭐⭐],False,False,True,...,23857029567770755,4368.77,299836.0,3322.0,32.0,6334.00,2023-08-01,4368.77,6334.00,1
3462,16038,923409341101559,23857029576390755,dynamic,[Change the Way You Shop],[Get summer-ready in styles with our high-qual...,[⭐⭐⭐⭐⭐],False,False,True,...,23857029576400755,2582.31,193128.0,1759.0,25.0,3812.50,2023-08-01,2582.31,3812.50,1
3463,16038,923409341101559,23857109574380755,dynamic,[Change the Way You Shop],[Embrace Effortless Elegance 🌸],[⭐⭐⭐⭐⭐],False,False,True,...,23857109574390755,1748.24,132285.0,2393.0,15.0,2054.61,2023-08-01,1748.24,2054.61,1


In [26]:
class A(ABC):
    @classmethod
    @abstractmethod
    def konj():
        pass

In [27]:
class B(A):
    def konj():
        print('hola')

In [28]:
B.konj()

hola


In [12]:
class B(A):
    pass

In [13]:
a = B()
a.konj()

b


In [3]:
DescriptiveDF.main in os.path.join('konj', DescriptiveDF.main)

True

In [ ]:
df = ftd

In [2]:
folder = 'proba1'
path = 'proba1/hopla.csv'
df = pd.DataFrame([[1, 0]])
save_csv_to_s3(df=df, path=path, add_global_folder=False)

In [3]:
bucket = 'creative-features'
copy_source = {
    "Bucket": bucket,
    "Key": 'proba1/hopla.csv',
}
s3_client.copy(copy_source, bucket, 'proba2/hopla.csv')

In [5]:
delete_from_s3(prefix='proba2/')

2023-08-18 10:01:34.741 | DEBUG    | src.utils.s3.delete_from_s3:delete_from_s3:15 - prefix = 'proba2/'


In [6]:
copy_on_s3(original_prefix=folder, target_prefix='proba2/', add_global_folder=False)

2023-08-18 10:01:49.897 | DEBUG    | src.utils.s3.utils:list_objects_from_prefix:45 - raw prefix = proba1
2023-08-18 10:01:49.903 | DEBUG    | src.utils.s3.copy_on_s3:copy_on_s3:26 - obj_path = 'proba1/hopla.csv', new_path = 'proba2/hopla.csv', creative-features


In [2]:
print(list_objects_from_prefix(prefix=path, add_global_folder=False))
delete_from_s3(prefix=path)
print(list_objects_from_prefix(prefix=path, add_global_folder=False))

2023-08-18 08:03:42.564 | DEBUG    | src.utils.s3.utils:list_objects_from_prefix:44 - raw prefix = prod


['prod/', 'prod/data/fb_images/', 'prod/data/fb_images/color_analysis/', 'prod/data/fb_images/color_analysis/final.csv', 'prod/data/fb_images/color_analysis/final_by_shop.csv', 'prod/data/fb_images/color_analysis/image.csv', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdac16-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdacd4-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdace8-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdacf2-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdacfc-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdad06-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdad10-e374-11ed-a595-66c93374e280.png', 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdad1a-e374-11ed

2023-08-18 08:04:49.393 | DEBUG    | src.utils.s3.utils:list_objects_from_prefix:44 - raw prefix = prod


[]


In [3]:
copy_on_s3(original_prefix='dev', copy_prefix='prod')

TypeError: copy_on_s3() got an unexpected keyword argument 'copy_prefix'

In [5]:
l = list_objects_from_prefix(prefix='prod/', add_global_folder=False)
l[8]

'prod/data/fb_images/color_analysis/images_1000px/images/a6bdacde-e374-11ed-a595-66c93374e280.png'

In [6]:
ta = 'prod/data/fb_images/color_analysis/images_1000px/images/a6bdacde-e374-11ed-a595-66c93374e280.png'

In [7]:
delete_from_s3(path=ta)

{'ResponseMetadata': {'RequestId': 'H0JCJQWXQQFMQVJ7',
  'HostId': 'm88AvNyg+7zki0B7M1NQRpAOUn1qni/2bX8BjiQx/GO+53AAHFCwjhNGrb+sJZgm7o9XzBWN1RY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'm88AvNyg+7zki0B7M1NQRpAOUn1qni/2bX8BjiQx/GO+53AAHFCwjhNGrb+sJZgm7o9XzBWN1RY=',
   'x-amz-request-id': 'H0JCJQWXQQFMQVJ7',
   'date': 'Fri, 18 Aug 2023 07:21:06 GMT',
   'x-amz-version-id': 'Jso_UnkZkUgUQcxHURvnVLDzqvsaZ_ie',
   'x-amz-delete-marker': 'true',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': 'Jso_UnkZkUgUQcxHURvnVLDzqvsaZ_ie'}

In [2]:
aha = get_conversion_rates_dict(db=db, account_model=FacebookAdAccount, shop_id = 44301396)
aha

SELECT facebook_ad_account.facebook_id AS facebook_ad_account_facebook_id, facebook_ad_account.currency AS facebook_ad_account_currency, anon_1.rate_from_usd AS anon_1_rate_from_usd 
FROM facebook_ad_account JOIN (SELECT currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd, rank() OVER (PARTITION BY currency_exchange_rate.code ORDER BY currency_exchange_rate.date DESC) AS date_rank 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON anon_1.code = facebook_ad_account.currency 
WHERE facebook_ad_account.shop_id IN (__[POSTCOMPILE_shop_id_1]) AND anon_1.date_rank = %(date_rank_1)s


{'57086910': 1.357765}

In [3]:
aha

{'57086910': 1.357765}

In [2]:
crud.currency_exchange_rate.query_current_rates(db=db)

SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS currency_exchange_rate_code, currency_exchange_rate.rate_from_usd AS currency_exchange_rate_rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC


In [8]:
df = pd.DataFrame()

In [10]:
getattr(df, 'sum')()

0.0

In [4]:
df = ping_target_and_performance(db=db, shop_id=shop_id, start_date=start_date, end_date=end_date, enum_to_value=True)

In [6]:
df.target

0      remarketing
1      remarketing
2      remarketing
3      remarketing
4      acquisition
          ...     
943    acquisition
944    acquisition
945    acquisition
946    remarketing
947    acquisition
Name: target, Length: 948, dtype: object

In [24]:
df = df[df.target == Target.remarketing]
df

,shop_id,account_id,adset_id,target,targeting,audience,interests,gender,age_groups,ad_id,spend,impr,clicks,purch,purch_value,year_month,spend_USD,purch_value_USD
0,44301396,57086910,6265489292658,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6270216041258,968.33,72098.0,737.0,57.0,5251.74,2022-08-01,713.162571,3867.838860
1,44301396,57086910,6265489292658,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6270216041258,1385.18,81919.0,957.0,43.0,5125.13,2022-09-01,1020.167227,3774.592225
2,44301396,57086910,6265489292658,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6270216041258,1839.94,108259.0,1276.0,143.0,22359.86,2022-10-01,1355.092109,16467.748861
3,44301396,57086910,6265489292658,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6270216041258,1913.82,72621.0,708.0,80.0,11620.13,2022-11-01,1409.503777,8558.076060
119,44301396,57086910,6284094837258,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6284094837458,13.66,676.0,73.0,0.0,0.00,2022-09-01,10.060414,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,44301396,57086910,6318012910458,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6318012909858,821.03,63646.0,629.0,32.0,2591.75,2023-05-01,604.678019,1908.790489
927,44301396,57086910,6318012910458,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6318012909858,0.00,0.0,0.0,0.0,0.00,2023-06-01,0.000000,0.000000
933,44301396,57086910,6336914364458,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6336914364258,193.37,24488.0,357.0,18.0,1742.20,2023-06-01,142.414514,1283.107858
934,44301396,57086910,6336914364458,Target.remarketing,"{'age_max': 65, 'age_min': 18, 'geo_locations'...",Audience.broad,[],Gender.both,"[AgeGroup.young, AgeGroup.middle, AgeGroup.old]",6336914364258,631.08,78041.0,927.0,152.0,16838.98,2023-07-01,464.782291,12401.691858


In [25]:
df.targeting.apply(lambda x: x.get('age_min')).describe()

count    138.0
mean      18.0
std        0.0
min       18.0
25%       18.0
50%       18.0
75%       18.0
max       18.0
Name: targeting, dtype: float64

In [18]:
df.age_groups

0      [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
1      [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
2      [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
3      [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
4      [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
                            ...                       
943    [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
944    [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
945    [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
946    [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
947    [AgeGroup.young, AgeGroup.middle, AgeGroup.old]
Name: age_groups, Length: 948, dtype: object

In [28]:
df = pd.DataFrame([[0, []], [2, [1, 2]]], columns=['a', 'b'])
df

,a,b
0,0,[]
1,2,"[1, 2]"


In [29]:
df.explode('b').dropna(axis=0, subset='b')

,a,b
1,2,1
1,2,2


In [74]:
data = db.query(FacebookAdset.targeting).filter(func.random() < 0.01).limit(10000).all()
data = [row.targeting for row in data]

In [65]:
i = np.random.randint(0, len(data)-1)
row = data[i]
print(row.get('age_min', None))
print(row.get('age_max', None))

18
36


In [75]:
df = pd.DataFrame([row for row in data if type(row) == dict])[['age_min', 'age_max']]

In [77]:
df['mid'] = df.apply(lambda row: (row.age_max+row.age_min)/2, axis=1)

In [79]:
df.age_min.describe()

count    6769.000000
mean       21.003398
std         6.160229
min        13.000000
25%        18.000000
50%        18.000000
75%        23.000000
max        65.000000
Name: age_min, dtype: float64

In [2]:
i1 = MyInterval(1, 2.5, right_border=Border.open)
i2 = MyInterval(2, 3)
i = i1 & i2

In [3]:
str(i)

'[2, 2.5>'

In [18]:
i.left_border

<Border.closed: 'closed'>

In [9]:
class AgeGroup(str, Enum):
    young = "young"
    middle = "middle"
    old = "old"

agegroup_interval_dict= {
    AgeGroup.young: MyInterval(18, 30),
    AgeGroup.middle: MyInterval(30, 50),
    AgeGroup.old: MyInterval(50, 100)
}

def get_agegroup(targeting: dict) -> list[AgeGroup]:
    age_min = targeting.get('age_min', 0)
    age_max = targeting.get('age_max', 100)
    age_interval = MyInterval(age_min, age_max)
    age_groups = []
    for agegroup, agegroup_interval in agegroup_interval_dict.items():
        if age_interval.mid in agegroup_interval or (age_interval & agegroup_interval).length / agegroup_interval.length > 0.3:
            age_groups.append(agegroup)
    return age_groups

In [16]:
def get_agegroup(targeting: dict):
    age_min = targeting.get('age_min', 0)
    age_max = targeting.get('age_max', 100)
    age_interval = MyInterval(age_min, age_max)
    age_groups = []
    for agegroup, agegroup_interval in agegroup_interval_dict.items():
        if age_interval.mid in agegroup_interval or (age_interval & agegroup_interval).length / agegroup_interval.length > 0.3:
            age_groups.append(agegroup)
    return age_groups
        

In [17]:
targeting = {'age_min': 18, 'age_max': 120}
get_agegroup(targeting)

[<AgeGroup.young: 'young'>, <AgeGroup.middle: 'middle'>, <AgeGroup.old: 'old'>]

In [14]:
interval([1, 2], [3, 4])

interval([1.0, 2.0], [3.0, 4.0])

In [3]:
type(np.nan)

float

In [30]:
3 - np.nan

nan

In [28]:
test_shop_phrases = ('anci', 'tatjana', 'nikola', 'test')
filters = [Shop.name.like('%'+ phrase + '%') for phrase in test_shop_phrases]
ah = db.query(Shop).filter(not_(or_(*filters))).all()
len(ah)

4320

In [2]:
BOOLEAN_TEXT_FEATURES

['cta',
 'user_addressing',
 'percentage',
 'free_shipping',
 'emoji',
 'hashtag',
 'starts_with_question',
 'price',
 'urgency',
 'fact_words',
 'link',
 'discount',
 'weasel_words']

In [7]:
df_old = ping_creative(db=db, shop_id=16038, start_date=start_date, end_date=end_date)

In [8]:
df_new = ping_creative2(db=db, shop_id=16038, start_date=start_date, end_date=end_date)